In [1]:
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
from langchain.chains import TransformChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from rich.pretty import pprint

# maybe use a cheaper model for the rewrite?

In [8]:
model = ChatBedrock(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs={"temperature": 0.99, 'max_tokens': 20000},
    guardrails={"guardrailIdentifier": "z0x1jv6cnbct", "guardrailVersion": "1"}
)

In [9]:
unchanged_prompt = PromptTemplate(
    input_variables=["prompt"], 
    template="{prompt}"
)

In [10]:
rewrite_prompt = PromptTemplate(
    template="""
act as an expert in prompt engineering who is skilled at 
getting the most useful responses possible from a generative AI model.

rewrite the following to be a more effective prompt.
the prompt should emphasize a succinct response.

return only the text of the rewritten prompt, without a preamble.
                                           
{prompt}
"""
)

In [11]:
mapper = PromptTemplate(
    input_variables=["content"], 
    template="{content}"
)

transform_chain = TransformChain(
    input_variables=["content"], 
    output_variables=["content"], 
    transform=lambda data: {'content': data['content'].content}
)

In [12]:
chain = RunnableParallel(
    original = unchanged_prompt | model | StrOutputParser(),
    rewrite = rewrite_prompt | model | transform_chain | mapper  | model | StrOutputParser()
)

In [13]:
response = chain.invoke(
    {
        "prompt": "is long distance space travel possible?"
    }
)

print('ORIGINAL:')
print(response['original'])
print('---------------------------------------')
print('REWRITE:')
print(response['rewrite'])

ORIGINAL:
Long distance space travel is theoretically possible, but there are significant technological and practical challenges that have not yet been fully overcome. Here are some key points about the possibility of long distance space travel:

- It is possible within the bounds of known physics and does not violate any fundamental laws of nature like Einstein's theory of relativity. However, it requires enormous amounts of energy.

- Chemical rockets like those used for current space missions are not practical for interstellar travel due to the massive amounts of propellant required over vast distances.

- Theorized propulsion methods that could enable practical interstellar travel include antimatter rockets, nuclear pulse propulsion, and hypothetical concepts like warp drives and wormholes. However, these remain speculative and have daunting engineering challenges.

- Travel to the nearest stars measured in light years would take centuries to millennia with current technology. Trav